# Lab 1 Part 2: Introduction to Computer Vision with OpenCV

**Student Name:**

**Student ID:**

**Date:** February 2026

UIR, ESIN

---

1. Introduction to OpenCV
2. Getting Started with OpenCV
3. Linear Filters
4. Non-Linear Filters {#nonlinear-filters}
5. Comprehensive Comparison: All Filters
6. Exercise: Filter Challenge

---

## 1. Introduction to OpenCV <a id='intro'></a>

**OpenCV** (Open Source Computer Vision Library) is one of the most widely used open-source libraries for computer vision and image processing tasks.

### Key Features:
- **Over 2,500 algorithms** for image processing, video analysis, object detection, and machine learning
- **Real-time processing** capabilities optimized for performance
- **Cross-platform** support (Windows, macOS, Linux, Android, iOS)
- **Multi-language** support (Python, C++, Java, MATLAB)
- **Hardware acceleration** with CUDA and OpenCL for GPU processing
- **Extensive documentation** and large community support

### Main Applications:
- Image and video processing
- Object detection and recognition
- Face detection and recognition
- Motion tracking and analysis
- Augmented reality
- Robotics and autonomous vehicles
- Medical imaging

Originally developed by Intel in 1999, OpenCV has become the de facto standard for traditional computer vision tasks.

---

## 2. Getting Started with OpenCV <a id='getting-started'></a>

### Installation

Install OpenCV using pip:

In [ ]:
# Install OpenCV (run this in your terminal or uncomment to run here)
# !pip install opencv-python opencv-contrib-python numpy matplotlib

### Basic Setup and Image Loading

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

# Configure matplotlib for better image display
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['image.cmap'] = 'gray'

print(f"OpenCV Version: {cv2.__version__}")

### Helper Function: Display Images

In [ ]:
def display_images(images: list, titles: list, cmap: str = 'gray', figsize: Tuple[int, int] = (15, 5)):
    """
    Display multiple images in a row.

    Args:
        images: List of images to display
        titles: List of titles for each image
        cmap: Colormap (default: 'gray')
        figsize: Figure size
    """
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)

    if n == 1:
        axes = [axes]

    for i, (img, title) in enumerate(zip(images, titles)):
        axes[i].imshow(img, cmap=cmap)
        axes[i].set_title(title, fontsize=12)
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

### Create Sample Images for Demonstration

In [ ]:
# Create a synthetic grayscale image with geometric shapes
img_clean = np.zeros((400, 400), dtype=np.uint8)
cv2.rectangle(img_clean, (50, 50), (150, 150), 255, -1)
cv2.circle(img_clean, (300, 100), 50, 200, -1)
cv2.rectangle(img_clean, (200, 250), (350, 350), 150, -1)

# Add Gaussian noise
gaussian_noise = np.random.normal(0, 25, img_clean.shape).astype(np.float32)
img_noisy_gaussian = np.clip(img_clean.astype(np.float32) + gaussian_noise, 0, 255).astype(np.uint8)

# Add salt-and-pepper noise
img_noisy_sp = img_clean.copy()
prob = 0.05
random_matrix = np.random.random(img_clean.shape)
img_noisy_sp[random_matrix < prob/2] = 0
img_noisy_sp[random_matrix > 1 - prob/2] = 255

display_images(
    [img_clean, img_noisy_gaussian, img_noisy_sp],
    ['Clean Image', 'Gaussian Noise', 'Salt-and-Pepper Noise']
)

## 3. Linear Filters

### What are Linear Filters?

Linear filters apply **convolution** operations where each output pixel is a **weighted sum** of input pixels in a neighborhood.

**General discrete form (2D convolution):**
$$
g[i,j] = \sum_{m=1}^{M} \sum_{n=1}^{N} f[m,n] \cdot h[i-m, j-n]
$$
**"Mask", "Kernel", or "Filter"** refers to the small matrix $h$.

where:
- $f[m,n]$: input image pixels (size $M \times N$)
- $h$: kernel (e.g., $3 \times 3$ or $5 \times 5$)
- $g[i,j]$: output image

### Properties
- **Linearity:** $f(a \cdot x_1 + b \cdot x_2) = a \cdot f(x_1) + b \cdot f(x_2)$
- **Shift invariance:** Same kernel applied everywhere
- **Superposition:** Sum of filtered inputs

### Applications
- Smoothing (Gaussian/mean filters)
- Edge detection (Sobel, Prewitt)
- Sharpening (Laplacian)
- Blurring


### 3.1 Box Filter (Average Filter)

The simplest linear filter that replaces each pixel with the **average** of its neighbors.

**Kernel (3×3):**
$$
K = \frac{1}{9} \begin{bmatrix}
1 & 1 & 1 \\
1 & 1 & 1 \\
1 & 1 & 1
\end{bmatrix}
$$

**Use case:** Simple noise reduction, but causes blurring


In [ ]:
# Apply box filter with different kernel sizes
box_3x3 = cv2.boxFilter(img_noisy_gaussian, -1, (3, 3))
box_7x7 = cv2.boxFilter(img_noisy_gaussian, -1, (7, 7))
box_15x15 = cv2.boxFilter(img_noisy_gaussian, -1, (15, 15))

display_images(
    [img_noisy_gaussian, box_3x3, box_7x7, box_15x15],
    ['Noisy Image', 'Box 3×3', 'Box 7×7', 'Box 15×15']
)

print("\n")
print("Note: Larger kernel size = more smoothing but more blurring")

### 3.2 Gaussian Filter

A **weighted average** where weights follow a Gaussian (normal) distribution. Gives more weight to nearby pixels.

**2D Gaussian function:**
$$
G(x, y) = \frac{1}{2\pi\sigma^2} e^{-\frac{x^2 + y^2}{2\sigma^2}}
$$

where $\sigma$ is the standard deviation controlling the spread.

**Example 5×5 Gaussian kernel (σ ≈ 1):**
$$
K = \frac{1}{256} \begin{bmatrix}
1 & 4 & 6 & 4 & 1 \\
4 & 16 & 24 & 16 & 4 \\
6 & 24 & 36 & 24 & 6 \\
4 & 16 & 24 & 16 & 4 \\
1 & 4 & 6 & 4 & 1
\end{bmatrix}
$$

**Advantages over box filter:**
- Better noise reduction
- Less blurring of edges
- No ringing artifacts


In [ ]:
# Apply Gaussian filter with different parameters
# Syntax: cv2.GaussianBlur(image, kernel_size, sigmaX)
# kernel_size must be odd numbers

gauss_small = cv2.GaussianBlur(img_noisy_gaussian, (5, 5), 1)
gauss_medium = cv2.GaussianBlur(img_noisy_gaussian, (9, 9), 2)
gauss_large = cv2.GaussianBlur(img_noisy_gaussian, (15, 15), 3)

display_images(
    [img_noisy_gaussian, gauss_small, gauss_medium, gauss_large],
    ['Noisy Image', 'Gaussian 5×5, σ=1', 'Gaussian 9×9, σ=2', 'Gaussian 15×15, σ=3']
)

### 3.3 Comparison: Box vs Gaussian Filter

In [ ]:
# Compare box and Gaussian filters with same kernel size
box_9x9 = cv2.boxFilter(img_noisy_gaussian, -1, (9, 9))
gauss_9x9 = cv2.GaussianBlur(img_noisy_gaussian, (9, 9), 0)

display_images(
    [img_noisy_gaussian, box_9x9, gauss_9x9],
    ['Original Noisy', 'Box Filter 9×9', 'Gaussian Filter 9×9']
)

print("\n")
print("Observation: Gaussian filter preserves edges better than box filter")

### 3.4 Custom Linear Filters with filter2D

You can create **custom kernels** for specific effects.

In [ ]:
# Custom kernel examples

# 1. Identity kernel (no change)
kernel_identity = np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 0, 0]], dtype=np.float32)

# 2. Sharpening kernel
kernel_sharpen = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]], dtype=np.float32)

# 3. Edge detection (Laplacian)
kernel_edge = np.array([[0, 1, 0],
                        [1, -4, 1],
                        [0, 1, 0]], dtype=np.float32)

# 4. Emboss
kernel_emboss = np.array([[-2, -1, 0],
                          [-1, 1, 1],
                          [0, 1, 2]], dtype=np.float32)

# Apply custom filters
img_identity = cv2.filter2D(img_clean, -1, kernel_identity)
img_sharpen = cv2.filter2D(img_clean, -1, kernel_sharpen)
img_edge = cv2.filter2D(img_clean, -1, kernel_edge)
img_emboss = cv2.filter2D(img_clean, -1, kernel_emboss)

display_images(
    [img_clean, img_sharpen, img_edge, img_emboss],
    ['Original', 'Sharpening', 'Edge Detection', 'Emboss']
)

print("\nKernels used:")
print("Sharpen:\n", kernel_sharpen)
print("\nEdge:\n", kernel_edge)
print("\nEmboss:\n", kernel_emboss)

## 4. Non-Linear Filters {#nonlinear-filters}

### What are Non-Linear Filters?

Non-linear filters do **not** follow the superposition principle. **Cannot be implemented using convolution.**

**Key characteristic:**
$$
f(a \cdot x_1 + b \cdot x_2) \neq a \cdot f(x_1) + b \cdot f(x_2)
$$

### Why Use Non-Linear Filters?
- **Better edge preservation** while reducing noise
- **Effective for specific noise types** (e.g., salt-and-pepper noise)
- **Can preserve image structure** better than linear filters

### Common Non-Linear Filters:
1. **Median filter** (order-statistic)
2. **Bilateral filter** (edge-preserving)  
3. **Morphological operations** (min/max filters)

### 4.1 Median Filter

Replaces each pixel with the **median** value of neighboring pixels.

**Algorithm:**
1. Extract neighborhood pixels
2. Sort them
3. Select the middle value

**Example (3×3 window):**
```
Neighborhood: [10, 12, 255, 15, 13, 14, 11, 16, 12]
Sorted: [10, 11, 12, 12, 13, 14, 15, 16, 255]
Median = 13
```

**Advantages:**
- **Excellent for salt-and-pepper noise** (impulse noise)
- **Preserves edges** better than linear filters
- Removes outliers effectively

In [ ]:
# Apply median filter to salt-and-pepper noise
median_3 = cv2.medianBlur(img_noisy_sp, 3)
median_5 = cv2.medianBlur(img_noisy_sp, 5)
median_7 = cv2.medianBlur(img_noisy_sp, 7)

# Compare with Gaussian filter on same image
gaussian_sp = cv2.GaussianBlur(img_noisy_sp, (5, 5), 0)

display_images(
    [img_noisy_sp, gaussian_sp, median_3, median_5],
    ['Salt-Pepper Noise', 'Gaussian 5×5', 'Median 3×3', 'Median 5×5']
)

print("\n")
print("Observation: Median filter is MUCH better for salt-and-pepper noise!")

### 4.2 Bilateral Filter

An **edge-preserving** smoothing filter that considers both:
1. **Spatial distance** (like Gaussian)
2. **Intensity difference** (range filter)

**Formula:**
$$
BF[I]_p = \frac{1}{W_p} \sum_{q \in S} G_{\sigma_s}(\|p - q\|) \cdot G_{\sigma_r}(|I_p - I_q|) \cdot I_q
$$

**Detailed Bilateral Filter Formula:**

$$
g[i,j] = \frac{1}{W_p} \sum_m \sum_n f[m,n] \, g_s(|i-m|,|j-n|) \, g_b(|f[m,n] - f[i,j]|)
$$

**Where:**
$$
g_s(m,n) = \frac{1}{2\pi\sigma_s^2} e^{-\frac{m^2+n^2}{2\sigma_s^2}}, \quad
g_b(k) = \frac{1}{\sqrt{2\pi}\sigma_b} e^{-\frac{k^2}{2\sigma_b^2}}
$$

$$
W_p = \sum_m \sum_n g_s(|i-m|,|j-n|) \, g_b(|f[m,n] - f[i,j]|)
$$

**Non-Linear Operation** (Cannot be implemented using convolution)


where:
- $G_{\sigma_s}$: Spatial Gaussian (distance weight)
- $G_{\sigma_r}$: Range Gaussian (intensity difference weight)  
- $W_p$: Normalization factor

**OpenCV Parameters:**
- `d`: Diameter of pixel neighborhood
- `sigmaColor`: Filter sigma in color space
- `sigmaSpace`: Filter sigma in coordinate space

**Key property:** Pixels with **similar intensities** are averaged; pixels with **different intensities** (edges) are preserved.


In [ ]:
# Apply bilateral filter
# Syntax: cv2.bilateralFilter(image, d, sigmaColor, sigmaSpace)

bilateral_1 = cv2.bilateralFilter(img_noisy_gaussian, 9, 75, 75)
bilateral_2 = cv2.bilateralFilter(img_noisy_gaussian, 9, 150, 150)

# Compare with Gaussian filter
gaussian_comp = cv2.GaussianBlur(img_noisy_gaussian, (9, 9), 0)

display_images(
    [img_noisy_gaussian, gaussian_comp, bilateral_1, bilateral_2],
    ['Noisy Image', 'Gaussian 9×9', 'Bilateral (σ=75)', 'Bilateral (σ=150)']
)

print("\n")
print("Observation: Bilateral filter smooths flat regions while preserving edges!")

### 4.3 Morphological Filters: Erosion and Dilation

**Min/Max filters** that operate on local neighborhoods.

**Erosion (Min filter):**
- Replaces pixel with **minimum** value in neighborhood
- Shrinks bright regions
- Removes small bright spots

**Dilation (Max filter):**
- Replaces pixel with **maximum** value in neighborhood
- Expands bright regions
- Fills small dark holes

In [ ]:
# Create structuring element (kernel)
kernel = np.ones((5, 5), np.uint8)

# Apply morphological operations
erosion = cv2.erode(img_clean, kernel, iterations=1)
dilation = cv2.dilate(img_clean, kernel, iterations=1)

# Opening: erosion followed by dilation (removes small bright spots)
opening = cv2.morphologyEx(img_noisy_sp, cv2.MORPH_OPEN, kernel)

# Closing: dilation followed by erosion (fills small dark holes)
closing = cv2.morphologyEx(img_noisy_sp, cv2.MORPH_CLOSE, kernel)

display_images(
    [img_clean, erosion, dilation],
    ['Original', 'Erosion (Min)', 'Dilation (Max)']
)

display_images(
    [img_noisy_sp, opening, closing],
    ['Salt-Pepper Noise', 'Opening', 'Closing']
)

---

## 5. Comprehensive Comparison: All Filters <a id='comparison-all'></a>

In [ ]:
# Apply all filters to Gaussian noise
results_gaussian = [
    (img_noisy_gaussian, 'Noisy (Gaussian)'),
    (cv2.boxFilter(img_noisy_gaussian, -1, (7, 7)), 'Box Filter'),
    (cv2.GaussianBlur(img_noisy_gaussian, (7, 7), 0), 'Gaussian Filter'),
    (cv2.medianBlur(img_noisy_gaussian, 7), 'Median Filter'),
    (cv2.bilateralFilter(img_noisy_gaussian, 9, 75, 75), 'Bilateral Filter')
]

fig, axes = plt.subplots(1, 5, figsize=(18, 4))
for i, (img, title) in enumerate(results_gaussian):
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(title, fontsize=11)
    axes[i].axis('off')
plt.suptitle('Comparison on Gaussian Noise', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# Apply all filters to salt-and-pepper noise
results_sp = [
    (img_noisy_sp, 'Noisy (Salt-Pepper)'),
    (cv2.boxFilter(img_noisy_sp, -1, (7, 7)), 'Box Filter'),
    (cv2.GaussianBlur(img_noisy_sp, (7, 7), 0), 'Gaussian Filter'),
    (cv2.medianBlur(img_noisy_sp, 7), 'Median Filter'),
    (cv2.bilateralFilter(img_noisy_sp, 9, 75, 75), 'Bilateral Filter')
]

fig, axes = plt.subplots(1, 5, figsize=(18, 4))
for i, (img, title) in enumerate(results_sp):
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(title, fontsize=11)
    axes[i].axis('off')
plt.suptitle('Comparison on Salt-and-Pepper Noise', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### Summary Table: When to Use Each Filter

| Filter | Type | Best For | Pros | Cons |
|--------|------|----------|------|------|
| **Box Filter** | Linear | Quick smoothing | Fast, simple | Blurs edges significantly |
| **Gaussian Filter** | Linear | General noise reduction | Smooth results, no artifacts | Blurs edges |
| **Median Filter** | Non-linear | Salt-and-pepper noise | Preserves edges, removes outliers | Slower, can lose details |
| **Bilateral Filter** | Non-linear | Edge-preserving smoothing | Excellent edge preservation | Computationally expensive |
| **Morphological** | Non-linear | Shape processing, binary images | Structural operations | Limited to specific tasks |

# 6. Exercise: Filter Challenge

**Challenge:** Find the **best filter combo** for Your images!

1. **Upload ANY image** (pet, landscape, object, face, art...)
2. Test **5 filter strategies**
3. Use **PSNR + visual quality** to rank them
4. **Explain** why the winner preserves faces best


### PSNR: Peak Signal to Noise Ratio

$$\text{PSNR} = 20 \log_{10} \left( \frac{255}{\sqrt{\text{MSE}}} \right)$$

**Higher = Better!**
| Range | Quality |
|-------|---------|
| >40dB | Excellent |
| 30-40 | Good |
| <25   | Poor |

In [ ]:
import cv2, numpy as np, matplotlib.pyplot as plt
from google.colab import files
%matplotlib inline

def psnr(ref, test):
    mse = np.mean((ref - test)**2)
    return 20*np.log10(255/np.sqrt(mse)) if mse else float('inf')

def add_sp_noise(img, salt_prob=0.08):
    noisy = img.copy()
    # Salt
    noisy[np.random.random(img.shape) < salt_prob] = 255
    # Pepper
    noisy[np.random.random(img.shape) < salt_prob/2] = 0
    return noisy

print("Upload a portrait/selfie:")
uploaded = files.upload()
filename = list(uploaded)[0]
face = cv2.imread(filename, 0)
face = cv2.resize(face, (400, 500))  # Portrait aspect

# Add realistic noise
noisy_face = add_sp_noise(face)

# 5 COMPETING STRATEGIES (your lecture filters!)
strategies = {
    '1. Box 7x7': cv2.blur(noisy_face, (7,7)),
    '2. Gaussian': cv2.GaussianBlur(noisy_face, (7,7), 2),
    '3. Median 7': cv2.medianBlur(noisy_face, 7),
    '4. Bilateral': cv2.bilateralFilter(noisy_face, 9, 50, 50),
    '5. Gaussian→Median': cv2.medianBlur(cv2.GaussianBlur(noisy_face, (5,5), 1), 5)
}

# BATTLE VISUALIZATION
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

results = {}
for i, (name, filtered) in enumerate(strategies.items()):
    psnr_val = psnr(face, filtered)
    results[name] = psnr_val
    axes[i].imshow(filtered, cmap='gray')
    axes[i].set_title(f'{name}\nPSNR: {psnr_val:.1f}dB', fontsize=11)
    axes[i].axis('off')

axes[5].imshow(face, cmap='gray')
axes[5].set_title('Original (Clean)', fontsize=11)
axes[5].axis('off')

plt.suptitle('Filter Face-Off: Which One Wins?', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# RANKING TABLE
print("\n RANKING:")
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
print("Strategy\t\tPSNR")
print("-"*30)
for name, score in sorted_results:
    print(f"{name:<16} {score:>6.1f}")


### Report Your Findings:
1. **Winner?** Which strategy recovered your image best?
2. **Why?** Explain using filter properties (median=salt-pepper, bilateral=edges, etc.)
3. **Parameter Play:** Try `cv2.bilateralFilter(..., 75, 75)` vs `(25, 25)`
4. Why did YOUR #1 filter win? Connect to theory! (e.g., Median kills salt/pepper, Bilateral saves edges, Gaussian smooths...)



---

**SUBMIT:** `File → Print → Save as PDF` to Connect following this format `YourName_ID_FiltersLab.pdf`
